In [1]:
import pandas as pd
import numpy as np
import requests

# Balancing energy prices

In [2]:


url = "https://api-baltic.transparency-dashboard.eu/api/v1/export?id=balancing_energy_prices&start_date=2023-01-01T00%3A00&end_date=2024-01-01T04%3A00&output_time_zone=EET&output_format=json&json_header_groups=0"

response = requests.get(url)

if response.status_code == 200:
    json_data = response.json()

    columns_info = json_data['data']['columns']
    column_headers = [f"{col['group_level_0']} {col['label']}" for col in columns_info]

    processed_data = []

    for entry in json_data['data']['timeseries']:
        from_datetime = pd.to_datetime(entry['from']).strftime('%Y-%m-%d %H:%M')
        
        row = {'from': from_datetime, 'to': entry['to']}
        row.update(dict(zip(column_headers, entry['values'])))
        processed_data.append(row)

    df = pd.DataFrame(processed_data)
    df_balancing_p=df.drop(columns= 'to')
else:
    print("Failed to fetch data")

df_balancing_p.columns = df_balancing_p.columns.str.lower().str.replace(' ', '_')



# Imbalance prices

In [3]:

url = 'https://api-baltic.transparency-dashboard.eu/api/v1/export?id=imbalance_prices&start_date=2023-01-01T00%3A00&end_date=2024-01-01T04%3A00&output_time_zone=EET&output_format=json&json_header_groups=0'

response = requests.get(url)

if response.status_code == 200:
    json_data = response.json()

    columns_info = json_data['data']['columns']
    column_headers = [f"{col['group_level_0']} {col['label']}" for col in columns_info]

    processed_data = []

    for entry in json_data['data']['timeseries']:
        from_datetime = pd.to_datetime(entry['from']).strftime('%Y-%m-%d %H:%M')
        
        row = {'from': from_datetime, 'to': entry['to']}
        row.update(dict(zip(column_headers, entry['values'])))
        processed_data.append(row)

    df = pd.DataFrame(processed_data)
    df_imbalance_p=df.drop(columns= 'to')
else:
    print("Failed to fetch data")

df_imbalance_p.columns = df_imbalance_p.columns.str.lower().str.replace(' ', '_')

#df_imbalance_p



# Imbalance volumes

In [4]:
import pandas as pd
import requests

url = 'https://api-baltic.transparency-dashboard.eu/api/v1/export?id=imbalance_volumes&start_date=2023-01-01T00%3A00&end_date=2024-01-01T04%3A00&output_time_zone=EET&output_format=json&json_header_groups=0'



response = requests.get(url)

if response.status_code == 200:
    json_data = response.json()

    columns_info = json_data['data']['columns']
    column_headers = [f"{col['group_level_0']} {col['label']}" for col in columns_info]

    processed_data = []

    for entry in json_data['data']['timeseries']:
        from_datetime = pd.to_datetime(entry['from']).strftime('%Y-%m-%d %H:%M')
        
        row = {'from': from_datetime, 'to': entry['to']}
        row.update(dict(zip(column_headers, entry['values'])))
        processed_data.append(row)

    df = pd.DataFrame(processed_data)
    df_imbalance_vol=df.drop(columns= 'to')
else:
    print("Failed to fetch data")

df_imbalance_vol.columns = df_imbalance_vol.columns.str.lower().str.replace(' ', '_')

#df_imbalance_vol


# mFRR normal activations

In [5]:


url = 'https://api-baltic.transparency-dashboard.eu/api/v1/export?id=normal_activations_mfrr&start_date=2023-01-01T00%3A00&end_date=2024-01-01T04%3A00&output_time_zone=EET&output_format=json&json_header_groups=0'
response = requests.get(url)

if response.status_code == 200:
    json_data = response.json()

    columns_info = json_data['data']['columns']
    column_headers = [f"{col['group_level_0']} {col['label']}" for col in columns_info]

    processed_data = []

    for entry in json_data['data']['timeseries']:
        # Convert 'from' to datetime and format it
        from_datetime = pd.to_datetime(entry['from']).strftime('%Y-%m-%d %H:%M')
        
        row = {'from': from_datetime, 'to': entry['to']}
        row.update(dict(zip(column_headers, entry['values'])))
        processed_data.append(row)

    df = pd.DataFrame(processed_data)
    mfrr_norm_act=df.drop(columns= 'to')
else:
    print("Failed to fetch data")

mfrr_norm_act.columns = mfrr_norm_act.columns.str.lower().str.replace(' ', '_')

#mfrr_norm_act


## Merge

In [6]:
dataframes = {
    'mfrr_norm_act': 'mfrr_act',
    'df_balancing_p': 'balancing_p',
    'df_imbalance_vol': 'imbalance_vol',
    'df_imbalance_p': 'imbalance_p'
}

for df_name, prefix in dataframes.items():
    df = globals()[df_name]
    
    if 'from' in df.columns:
        df.rename(columns={'from': 'datetime'}, inplace=True)
    
    df.columns = [col if col == 'datetime' else prefix + '_' + col for col in df.columns]

#mfrr_norm_act.head()


In [7]:
dataframe_names = ['mfrr_norm_act', 'df_balancing_p', 'df_imbalance_vol', 'df_imbalance_p']

merged_df = globals()[dataframe_names[0]]

for df_name in dataframe_names[1:]:
    merged_df = pd.merge(merged_df, globals()[df_name], on='datetime', how='inner')

#merged_df

In [8]:
merged_df['datetime'] = pd.to_datetime(merged_df['datetime'])

merged_df['datetime'] = merged_df['datetime'] + pd.to_timedelta(2, unit='h')

merged_df[:-4]

,datetime,mfrr_act_baltics_upward,mfrr_act_baltics_downward,mfrr_act_estonia_upward,mfrr_act_estonia_downward,mfrr_act_latvia_upward,mfrr_act_latvia_downward,mfrr_act_lithuania_upward,mfrr_act_lithuania_downward,mfrr_act_finland_upward,...,balancing_p_latvia_downward,balancing_p_lithuania_upward,balancing_p_lithuania_downward,imbalance_vol_baltics_none,imbalance_vol_estonia_none,imbalance_vol_latvia_none,imbalance_vol_lithuania_none,imbalance_p_estonia_none,imbalance_p_latvia_none,imbalance_p_lithuania_none
0,2023-01-01 00:00:00,0.000,29.767,0.000,0.000,0.0,0.0,0.000,29.767,0.0,...,-321.00,-321.00,-321.00,145.652,16.096,-0.939,130.495,-309.49,-309.49,-309.49
1,2023-01-01 01:00:00,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,...,-6.72,-6.72,-6.72,95.722,-37.693,12.697,120.718,4.79,4.79,4.79
2,2023-01-01 02:00:00,0.000,73.333,0.000,0.000,0.0,0.0,0.000,73.333,0.0,...,-321.00,-321.00,-321.00,211.622,67.414,-12.370,156.578,-309.49,-309.49,-309.49
3,2023-01-01 03:00:00,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,...,-6.00,-6.00,-6.00,180.717,70.903,-9.104,118.918,5.51,5.51,5.51
4,2023-01-01 04:00:00,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,...,-6.00,-6.00,-6.00,164.364,77.669,3.144,83.551,5.51,5.51,5.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00,0.000,9.417,0.000,9.417,0.0,0.0,0.000,0.000,0.0,...,10.10,10.10,10.10,30.471,72.878,-11.558,-30.849,17.74,17.74,17.74
8756,2023-12-31 20:00:00,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,...,23.86,23.86,23.86,-161.120,106.282,8.694,-276.096,31.50,31.50,31.50
8757,2023-12-31 21:00:00,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,...,-6.25,-6.25,-6.25,67.701,118.840,11.112,-62.251,1.39,1.39,1.39
8758,2023-12-31 22:00:00,40.000,0.000,0.000,0.000,0.0,0.0,40.000,0.000,37.5,...,75.00,75.00,75.00,-73.847,13.914,8.898,-96.659,67.36,67.36,67.36


In [9]:

df_baltic_imbalance_prices = merged_df.loc[:, 
    (merged_df.columns.str.contains('datetime', case=False)) |
    (merged_df.columns.str.contains('imbalance_p_estonia_none', case=False)) |
    (merged_df.columns.str.contains('mfrr', case=False) & merged_df.columns.str.contains('baltic', case=False)) |
    (merged_df.columns.str.contains('imbalance_p_estonia_none', case=False)) |
    (merged_df.columns.str.contains('balancing_p_estonia_upward', case=False)) |
    (merged_df.columns.str.contains('balancing_p_estonia_downward', case=False))
]


df_baltic_imbalance_prices['mfrr_price_approx_upward'] = np.where(
                                                            df_baltic_imbalance_prices['mfrr_act_baltics_upward'] > 0,
                                                            df_baltic_imbalance_prices['balancing_p_estonia_upward'],
                                                            0  # Set to max if the ratio is 0
                                                        )


df_baltic_imbalance_prices['mfrr_price_approx_downward'] = np.where(
                                                            df_baltic_imbalance_prices['mfrr_act_baltics_downward'] > 0,
                                                            df_baltic_imbalance_prices['balancing_p_estonia_downward'],
                                                            0  # Set to max if the ratio is 0
                                                        )



df_nec_cols = df_baltic_imbalance_prices[['datetime', 'imbalance_p_estonia_none', 'mfrr_act_baltics_upward', 'mfrr_act_baltics_downward', 'mfrr_price_approx_upward', 'mfrr_price_approx_downward']]


<ipython-input-9-bc47c6e90b1f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_baltic_imbalance_prices['mfrr_price_approx_upward'] = np.where(
<ipython-input-9-bc47c6e90b1f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_baltic_imbalance_prices['mfrr_price_approx_downward'] = np.where(


In [10]:
df_nec_cols.to_csv('btd_nec.csv', sep=';', index=False)

# Elering data

In [11]:

url = "https://dashboard.elering.ee/api/nps/price?start=2022-12-31T23%3A00%3A00Z&end=2023-12-31T22%3A00%3A00Z"

response = requests.get(url)

if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print("Data retrieved successfully.")
else:
    print("Failed to retrieve data. Status code:", response.status_code)


price_data = data['data']['ee']
df_da = pd.DataFrame(price_data)
df_da['datetime'] = pd.to_datetime(df_da['timestamp'], unit='s')
df_da = df_da.rename(columns = {'price': 'DA_price'})
df_da['datetime'] = df_da['datetime'] + pd.Timedelta(hours=1)
df_da.tail()



Data retrieved successfully.


,timestamp,DA_price,datetime
8755,1704045600,59.99,2023-12-31 19:00:00
8756,1704049200,40.99,2023-12-31 20:00:00
8757,1704052800,53.83,2023-12-31 21:00:00
8758,1704056400,55.49,2023-12-31 22:00:00
8759,1704060000,40.01,2023-12-31 23:00:00


In [12]:
df_da.to_csv('elering_da_data.csv', sep=';', index=False)